In [2]:
import pandas as pd
fh = '../files/tickets-gen-all.csv'
df = pd.read_csv(fh, index_col=0, parse_dates=['created', 'opened_at', 'updated_on', 'resolved'])
df.shape

(75724, 22)

In [3]:
df.columns

Index(['created', 'caller', 'opened_at', 'opened_by', 'priority', 'state',
       'category', 'subcategory', 'assignment_group', 'bpl_location',
       'nyp_locaiton', 'resolved', 'assigned_to', 'system', 'closed_at',
       'closed_by', 'mat_source', 'reassignment_count', 'reopen_count',
       'resolved_by', 'updated_on', 'updated_by'],
      dtype='object')

##### cataloging active tickets only

In [4]:
cadf = df[((df['category'] == 'Cataloging') | (df['assignment_group'] == 'BKOPS CAT')) & ((df['state'] != 'Closed') & (df['state'] != 'Resolved'))]

## Cataloging tickets requiring category change (diff dept)

In [5]:
df['category'].unique()

array(['Cataloging', 'Acquisitions', 'Selection', 'Logistics',
       'Offsite - ReCAP', 'Collections Processing', nan], dtype=object)

In [6]:
df['assignment_group'].unique()

array(['BKOPS CAT', 'BKOPS ACQ', 'BKOPS CPRC', 'BKOPS LOG', 'BKOPS RECAP',
       'BKOPS SEL', 'IT ILS', 'IT Branch Library Support', nan,
       'BKOPS ITIL', 'BKOPS LOG - MyLibNYC'], dtype=object)

In [7]:
cat_change_df = df[(df['category'] == 'Cataloging') & ((df['assignment_group'] != 'BKOPS CAT') & (df['assignment_group'].notnull()))]

In [8]:
cat_change_df.shape[0]

587

In [9]:
cat_change_df_active = cat_change_df[(cat_change_df['state'] != 'Closed') & (cat_change_df['state'] != 'Resolved')]

In [10]:
print(f'# of tickets: {cat_change_df_active.shape[0]}')

# of tickets: 34


## Awaiting User & Vendor tickets

In [11]:
awaiting_df = cadf[((cadf['state'] == 'Awaiting Vendor') | (cadf['state'] == 'Awaiting User Info')) & (cadf['created'] < '2020-01-01')]

In [12]:
awaiting_df['state'].unique()

array(['Awaiting User Info', 'Awaiting Vendor'], dtype=object)

In [13]:
print(f'# of tickets: {awaiting_df.shape[0]}')

# of tickets: 748


### NEW tickets backlog (older than mid February 2020)

In [14]:
new_backlog_df = cadf[(cadf['state'] == 'New') & (cadf['assigned_to'].isnull())]

In [15]:
new_backlog_df.shape[0]

384

In [16]:
for lib, ldf in new_backlog_df.groupby('system'):
    print(lib, f'# of tickets: {ldf.shape[0]}')

BPL Circulating # of tickets: 193
NYPL Circulating # of tickets: 147
NYPL Research # of tickets: 44


## Active CAT tickets

In [17]:
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

caadf = cadf[cadf['state'] == 'Active']

for lib, ldf in caadf.groupby('system'):
    staff_df = pd.DataFrame(columns=['staff', 'year', 'tickets'])
    for staff, sdf in ldf.groupby('assigned_to'):
        d = dict()
        for y, ydf in sdf.groupby(sdf['created'].map(lambda x: x.year)):
            d[y]={'staff': staff, 'year': y, 'tickets': ydf.shape[0]}
        for y in years:
            if y in d.keys():
                 staff_df = staff_df.append(d[y], ignore_index=True)
            else:
                staff_df = staff_df.append({'staff': staff, 'year': y, 'tickets': 0}, ignore_index=True)
                
    staff_df.to_csv(f'../data-display/{lib}-active-tickets-by-staff.csv', index=False)
    

#### active by library per year

In [18]:
lib_out_df = pd.DataFrame(columns=['library', 'year', 'tickets'])
for lib, ldf in caadf.groupby('system'):
    d = dict()
    for y, ydf in ldf.groupby(ldf['created'].map(lambda x: x.year)):
        d[y]={'library': lib, 'year': y, 'tickets': ydf.shape[0]}
    for y in years:
        if y in d.keys():
             lib_out_df = lib_out_df.append(d[y], ignore_index=True)
        else:
            lib_out_df = lib_out_df.append({'library': lib, 'year': y, 'tickets': 0}, ignore_index=True)
lib_out_df.to_csv('../data-display/cat-active-tickets-per-lib-timeline.csv', index=False)

#### Active categories

In [23]:
cat_out_df = pd.DataFrame(columns=['subcategory', 'tickets'])
for cat, cdf in caadf.groupby('subcategory'):
    print(cat, cdf.shape[0])
    cat_out_df = cat_out_df.append(
        dict(
            subcategory=cat,
            tickets=cdf.shape[0]
        ),
        ignore_index=True
    )
cat_out_df.head()

Barcode sticker request (Research centers only) 2
Barcodes for circulating materials 5
Call number error 362
Call number missing 25
Cataloging Request for a Free Internet Resource 18
Cataloging error 202
Collection HQ 10
Damaged material / missing pieces 3
Duplicate records 1
Electronic resource 128
Holds fulfillment--NYPL only 7
Item attached to wrong bib record 56
Item not linked 83
Item record problem 41
Leased items 2
Location code error 11
OCLC Holdings 6
Online Catalog 47
Other 215
Spine labels 69
Status of request 1
Volume record needed 4


,subcategory,tickets
0,Barcode sticker request (Research centers only),2
1,Barcodes for circulating materials,5
2,Call number error,362
3,Call number missing,25
4,Cataloging Request for a Free Internet Resource,18


In [25]:
cat_out_df.to_csv('../data-display/cat-active-by-category.csv', index=False)

In [27]:

for lib, ldf in caadf.groupby('system'):
    lib_out_df = pd.DataFrame(columns=['category', 'tickets'])
    for cat, cdf in ldf.groupby('subcategory'):
        lib_out_df = lib_out_df.append(
            dict(
                category=cat,
                tickets=cdf.shape[0]
            ),
            ignore_index=True)
        
    lib_out_df.to_csv(f'../data-display/{lib}-active-tickets-by-category.csv', index=False)